# Initializing the Sim Manager Object

This notebook illustrates the how to work with the GPUDrive Sim.

In [29]:
# Import the gpudrive library
import gpudrive

import torch
import jax

### A basic sim object with default parameters

In [2]:
# Basic initialization of the Sim object 
sim = gpudrive.SimManager(
    exec_mode=gpudrive.madrona.ExecMode.CPU, # Specify the execution mode
    gpu_id=0, # If using GPU, specify the GPU ID
    auto_reset=False, # If True, the simulation will reset after the episode ends
    num_worlds=1, # Initializing only one world
    json_path='../nocturne_data', # Path to the json file
    params = gpudrive.Parameters() # Default params 
)

Sim exports `reset()` and `step()` 

In [5]:
sim.reset(0) # Reset the world with index 0
sim.step() # Steps through all the worlds in parallel

To get any tensor from the sim, we need to call the specific `tensor()` method followed by `to_torch()` or `to_jax()`

In [13]:
observation_tensor = sim.self_observation_tensor().to_torch()
print(observation_tensor.shape), print(observation_tensor.device)

torch.Size([1, 50, 6])
cpu


(None, None)

Or alternatively

In [17]:
observation_tensor_jax = sim.self_observation_tensor().to_jax()
print(observation_tensor_jax.shape), print(observation_tensor_jax.devices())

(1, 50, 6)
{CpuDevice(id=0)}


(None, None)

Available tensor exports and methods on the sim object

In [39]:
for attr in dir(sim):
    if not attr.startswith('_'):
        print(attr)

absolute_self_observation_tensor
action_tensor
agent_roadmap_tensor
bicycle_model_tensor
controlled_state_tensor
depth_tensor
done_tensor
lidar_tensor
map_observation_tensor
partner_observations_tensor
reset
reset_tensor
reward_tensor
rgb_tensor
self_observation_tensor
shape_tensor
step
steps_remaining_tensor
valid_state_tensor


Check the number of agents and roads in each world

In [26]:
shape_tensor = sim.shape_tensor().to_jax()
print("Shape tensor has a shape of (Num Worlds, 2): ", shape_tensor.shape)

# The shape tensor is a 2D tensor with the first dimension being the number of worlds and the second dimension being the shape of the world
for i in range(shape_tensor.shape[0]):
    print("World ", i, " has ", shape_tensor[i][0], " VALID agents and ", shape_tensor[i][1], " VALID road objects")

Shape tensor has a shape of (Num Worlds, 2):  (1, 2)
World  0  has  42  VALID agents and  4421  VALID road objects


Check how many agents are controlled by Bicycle Model

In [35]:
controlled_state_tensor = sim.controlled_state_tensor().to_torch()
print("Controlled state tensor has a shape of (Num Worlds, Total Num Agents, 1): ", controlled_state_tensor.shape)

print("Controlled state tensor: ", controlled_state_tensor[0].squeeze())
print("Agent indices in the World 0 where the agents are controlled: ", torch.where(controlled_state_tensor[0])[0])

Controlled state tensor has a shape of (Num Worlds, Total Num Agents, 1):  torch.Size([1, 50, 1])
Controlled state tensor:  tensor([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0], dtype=torch.int32)
Agent indices in the World 0 where the agents are controlled:  tensor([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 41])


Check the current actions tensor

In [41]:
actions_tensor = sim.action_tensor().to_torch()
print("Actions tensor has a shape of (Num Worlds, Total Num Agents, 2): ", actions_tensor.shape)

print("Current actions for agent 0 in World 0: ", actions_tensor[0][0])

Actions tensor has a shape of (Num Worlds, Total Num Agents, 2):  torch.Size([1, 50, 3])
Current actions for agent 0 in World 0:  tensor([0., 0., 0.])


To set the action for an agent, we simply copy the actions from another tensor or set it directly

In [47]:
actions_tensor = sim.action_tensor().to_torch()

print("Setting ones for all agents in all worlds")
actions = torch.full(actions_tensor.shape, 1.0)
actions_tensor.copy_(actions)

print("Actions tensor after setting all actions to 1: ", actions_tensor[0][0])

# Call the step function to apply the actions
sim.step()

Setting ones for all agents in all worlds
Actions tensor after setting all actions to 1:  tensor([1., 1., 1.])


### Inspecting the default parameters of the sim

In [4]:
params = gpudrive.Parameters()

print("Parameters:")
for attr in dir(params):
    if not attr.startswith('__'):
        value = getattr(params, attr)
        print(f"{attr:20}: {value}")
        if attr == 'rewardParams':
            print("  Reward Parameters:")
            reward_params = getattr(params, attr)
            for attr2 in dir(reward_params):
                if not attr2.startswith('__'):
                    value2 = getattr(reward_params, attr2)
                    print(f"    {attr2:18}: {value2}")


Parameters:
collisionBehaviour  : gpudrive.CollisionBehaviour.AgentStop
datasetInitOptions  : gpudrive.DatasetInitOptions.FirstN
maxNumControlledVehicles: 10000
observationRadius   : 0.0
polylineReductionThreshold: 0.0
rewardParams        : <gpudrive.RewardParams object at 0x77728ace3cf0>
  Reward Parameters:
    distanceToExpertThreshold: 0.0
    distanceToGoalThreshold: 0.0
    rewardType        : gpudrive.RewardType.DistanceBased


To set the parameters of the Sim, just fill in the values for each attr of the param object. 

In [48]:
reward_params = gpudrive.RewardParams()
reward_params.rewardType = gpudrive.RewardType.DistanceBased
reward_params.distanceToGoalThreshold = 1.0
reward_params.distanceToExpertThreshold = 1.0 

# Initialize Parameters
params = gpudrive.Parameters()
params.polylineReductionThreshold = 1.0
params.observationRadius = 100.0
params.datasetInitOptions = gpudrive.DatasetInitOptions.RandomN
params.collisionBehaviour = gpudrive.CollisionBehaviour.Ignore
params.maxNumControlledVehicles = 10
params.rewardParams = reward_params

The params object can simply be passed to the sim constructor

In [50]:
# Basic initialization of the Sim object 
sim = gpudrive.SimManager(
    exec_mode=gpudrive.madrona.ExecMode.CPU, # Specify the execution mode
    gpu_id=0, # If using GPU, specify the GPU ID
    auto_reset=False, # If True, the simulation will reset after the episode ends
    num_worlds=1, # Initializing only one world
    json_path='../nocturne_data', # Path to the json file
    params = params 
)

## Running through one episode of the sim

In [57]:
num_worlds = 1

for i in range(num_worlds):
    sim.reset(i)

actions_shape = sim.action_tensor().to_torch().shape
dones = sim.done_tensor().to_torch()
while not torch.all(sim.done_tensor().to_torch()):
    obs, rews, dones = sim.self_observation_tensor().to_torch(), sim.reward_tensor().to_torch(), sim.done_tensor().to_torch()
    actions = torch.rand(actions_shape)
    sim.action_tensor().to_torch().copy_(actions)
    sim.step()